In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from matplotlib.lines import Line2D
from scipy import stats

In [2]:
col_names = ['rid', 'topic', 'query', 'value', 'wid', 'batch', 'duration', 'EndDate', 'query_mean']
dtypes = {'rid': str, 'topic': int, 'query': str, 'value': int, 'wid': str, 'batch': str, 'duration': int,
          'query_mean': float}
ranks_df = pd.read_csv('data/ranks_df_long.csv', index_col='rid', names=col_names, dtype=dtypes, header=0)
rates_df = pd.read_csv('data/ratings_df_long.csv', index_col='rid', names=col_names, dtype=dtypes, header=0)
ranks_df['value'] = ranks_df['value'].max() - ranks_df['value'] + 1

In [3]:
comb_df = pd.concat([ranks_df.assign(method='Ranking'), rates_df.assign(method='Rating')]).sort_values(
    ['topic', 'method', 'query_mean', 'value'])
comb_df['value'].clip(lower=1, inplace=True)  # clipping all ratings to start from 1

# calc std error
print(comb_df.groupby(['method', 'query'])['value'].mean().loc[('Rating', 'schizophrenia')] +
      comb_df.groupby(['method', 'query'])['value'].std().loc[('Ranking', 'schizophrenia')])

# calc ci's with seaborn
print(ranks_df.loc[ranks_df['topic'] == 213].groupby('query')['value'].apply(sns.algorithms.bootstrap).apply(
    sns.utils.ci).apply(lambda x: x[1] - x[0]))
print(rates_df.loc[rates_df['topic'] == 213].groupby('query')['value'].apply(sns.algorithms.bootstrap).apply(
    sns.utils.ci).apply(lambda x: x[1] - x[0]))

3.5652800705467813
query
carpal tunnel                     0.433333
carpal tunnel syndrome            0.377778
carpal tunnel syndrome wiki       0.444722
cts syndrome                      0.288889
what is carpal tunnel syndrome    0.388889
Name: value, dtype: float64
query
carpal tunnel                     0.465753
carpal tunnel syndrome            0.410959
carpal tunnel syndrome wiki       0.410959
cts syndrome                      0.493151
what is carpal tunnel syndrome    0.397260
Name: value, dtype: float64


In [4]:
print(
    f"Overall average time {comb_df['duration'].mean() // 60:.0f} minutes and {comb_df['duration'].mean() % 60:.0f} seconds")
print(f"with SD of {comb_df['duration'].std() // 60:.0f} minutes and {comb_df['duration'].std() % 60:.0f} seconds")

Overall average time 5 minutes and 17 seconds
with SD of 3 minutes and 52 seconds


In [5]:
print(f"Total of {comb_df['wid'].nunique()} workers submitted the filtered HITs")
print(f"Every worker did {comb_df.groupby('wid')['batch'].nunique().mean():.4g} HITs on average")

Total of 267 workers submitted the filtered HITs
Every worker did 1.94 HITs on average


In [6]:
comb_df.loc[comb_df['topic'].isin({213, 286})]

,topic,query,value,wid,batch,duration,EndDate,query_mean,method
rid,,,,,,,,,
48775020688,213,carpal tunnel syndrome,2,AH56J7I291XL7,red,246,2022-02-14 10:15:50,1.911111,Ranking
60832753977,213,carpal tunnel syndrome,2,A4D99Y82KOLC8,red,152,2022-02-14 14:30:47,1.911111,Ranking
99458784954,213,carpal tunnel syndrome,2,A13FTIHWW9GE8X,red,242,2022-02-14 14:07:38,1.911111,Ranking
12734195590,213,carpal tunnel syndrome,3,A1I0DV4B4MFQCL,red,286,2022-02-14 13:59:06,1.911111,Ranking
17248459875,213,carpal tunnel syndrome,3,A12R2U6TBB3OOG,red,299,2022-02-10 01:36:09,1.911111,Ranking
...,...,...,...,...,...,...,...,...,...
893638237006,286,drug treatment schizophrenia,5,A3FKXJVH0K2E3E,green,170,2022-02-14 11:44:18,4.693182,Rating
908844146155,286,drug treatment schizophrenia,5,A5NE8TWS8ZV7B,green,225,2022-02-14 11:59:07,4.693182,Rating
944288367126,286,drug treatment schizophrenia,5,A1X1SOI48T1C6E,green,204,2022-02-10 13:15:09,4.693182,Rating


In [7]:
unique_user_queries = pd.read_csv('data/unique_normalized_user_queries.csv', header=None, names=['qid', 'query'],
                                  index_col='query')
unique_user_queries = unique_user_queries.assign(rid=unique_user_queries.qid.str.rsplit('-', 1).str[1])
unique_user_queries

FileNotFoundError: [Errno 2] No such file or directory: 'data/unique_normalized_user_queries.csv'

In [ ]:
k = 10
ndcg_df = pd.read_csv(f'data/PL2.DFR.SD-survey.ndcg@{k}', sep='\t', header=None, names=['qid', f'nDCG@{k}'],
                      index_col='qid')
canary_qids = ndcg_df.loc[ndcg_df.index.str.endswith('-0')].index
# ndcg_df.index = ndcg_df.index.str.strip()
# ndcg_df.shape
users_ndcg_df = pd.read_csv(f'data/user_queries/PL2.DFR.SD.ndcg@{k}', sep='\t', header=None, names=['qid', f'nDCG@{k}'],
                            index_col='qid')
users_ndcg_df = users_ndcg_df.loc[unique_user_queries['qid']]
users_ndcg_df

In [ ]:
qdf = pd.read_csv('data/QueriesSurvey.csv', header=None, names=['qid', 'query'], index_col='qid').applymap(str.strip)
qdf.index = qdf.index.str.strip()
qdf = qdf.merge(ndcg_df, left_index=True, right_index=True)
qdf['topic'] = qdf.index.str.split('-').str[0]
qdf

In [ ]:
ranks_sr = ranks_df.groupby(['query'])['value'].mean()
rates_sr = rates_df.groupby(['query'])['value'].mean()
rates_sr.sort_values()

In [ ]:
qdf = qdf.assign(avg_rate=qdf['query'].map(rates_sr), avg_rank=qdf['query'].map(ranks_sr))
qdf_filtered = qdf.loc[~qdf.index.isin(canary_qids)].rename(
    columns={'topic': 'Topic', 'avg_rate': 'Rating', 'avg_rank': 'Ranking'})
qdf_filtered.head()
print('Pearson:')
print(qdf_filtered.corr(method='pearson').to_markdown(floatfmt='.2f'))
print(f"{stats.pearsonr(qdf_filtered['Rating'], qdf_filtered['Ranking'])[1]:.1g}")
print(f"{stats.pearsonr(qdf_filtered['Rating'], qdf_filtered['nDCG@10'])[1]:.1g}")
print(f"{stats.pearsonr(qdf_filtered['Ranking'], qdf_filtered['nDCG@10'])[1]:.1g}")

print('\nSpearman:')
print(qdf_filtered.corr(method='spearman').to_markdown(floatfmt='.2f'))
print('\nKendall:')
print(qdf_filtered.corr(method='kendall').to_markdown(floatfmt='.2f'))


In [ ]:
sns.set_style("whitegrid")

# _df = comb_df.loc[comb_df['topic'].isin({213, 223, 253, 286})]
_df = comb_df
g = sns.catplot(data=_df, x='value', y='query', hue='method', kind='point', col='topic',
                hue_order=['Ranking', 'Rating'], scale=0.8, errwidth=1.5, capsize=0.15, col_wrap=3,
                height=3, aspect=3, sharey=False, sharex=True, dodge=0.27, ci='sd', join=False, margin_titles=False,
                legend=False, legend_out=False)

for ax in g.axes.flat:
    ax.minorticks_on()
    # ax.tick_params(axis='y', which='major', left=False, pad=2)
    ax.tick_params(axis='y', which='minor', left=False)
    ax.tick_params(axis='x', which='minor', bottom=True)
    ax.tick_params(axis='x', which='major', bottom=True, labelbottom=True)

for ax in g.axes.flat:
    ax.set_ylabel(None)
    ax.set_xlabel(None)
    ax.set_xlabel("User's mean")

g.map(plt.grid, b=True, which='major', axis='x', zorder=1, linewidth=1, ms=0, alpha=0.6)
g.map(plt.grid, b=True, which='major', axis='y', zorder=1, linewidth=1, ms=0, alpha=0.6)
# g.map(plt.grid, b=True, which='minor', axis='x', zorder=0, alpha=0.3, linewidth=0.5, ms=0)

for ax in g.axes.flat:
    ax.legend(loc='upper left')
    # ax.set_title(None)

for l in ax.lines + ax.collections:
    l.set_zorder(20)

g.set_titles("Topic {col_name} User's Mean")

plt.savefig('plots/point_mean_ci.pdf', dpi=300, bbox_inches='tight')

In [ ]:
ranks_sr.corr(rates_sr, method='pearson')

In [ ]:
all_user_queries = pd.read_csv('data/all_normalized_user_queries.csv', header=None, names=['qid', 'user_query']).assign(
    topic=lambda x: x.qid.apply(lambda y: y.split('-')[0])).sort_values('qid')
all_user_queries = all_user_queries.assign(rid=all_user_queries.qid.str.rsplit('-', 1).str[1])
all_user_queries = all_user_queries.assign(method=all_user_queries.qid.str.split('-').str[1].str.capitalize())
all_user_queries = all_user_queries.assign(topic=all_user_queries.qid.str.split('-').str[0].astype(int))
# filter only the accepted queries
all_user_queries = all_user_queries.loc[all_user_queries['rid'].isin(comb_df.index)]
all_user_queries['ref_qid'] = all_user_queries['user_query'].apply(lambda x: unique_user_queries.loc[x, 'qid'])
all_user_queries[f'user_nDCG@{k}'] = all_user_queries['ref_qid'].apply(lambda x: users_ndcg_df.loc[x, f'nDCG@{k}'])
all_user_queries.set_index(['method', 'topic', 'rid'])
all_user_queries

In [ ]:
comb_df = comb_df.assign(qid=comb_df['query'].map(qdf.reset_index().set_index('query')['qid']))
comb_df

In [ ]:
_df = qdf_filtered.melt(id_vars=[f'nDCG@{k}', 'Topic'], value_vars=['Ranking', 'Rating'],
                        value_name="Users' mean").sort_values('Topic').rename(columns={f'nDCG@{k}': f'NDCG@{k}'})
_rate_df = _df.loc[_df['variable'] == 'Rating']
_rank_df = _df.loc[_df['variable'] == 'Ranking']

x = "Users' mean"
y = f'NDCG@{k}'
markers = list(Line2D.filled_markers)
for m in {'8', 'h', 'H'}:
    markers.remove(m)

_col_wrap = None
g = sns.lmplot(data=_df, y=y, x=x, hue='Topic', col='variable', height=2.8, aspect=1.7, ci=None, fit_reg=False,
               markers=markers, scatter_kws=dict(s=25), palette="icefire", facet_kws=dict(sharey=False),
               col_wrap=_col_wrap, col_order=['Ranking', 'Rating'])

_rank_reg_res = stats.linregress(_rank_df[x], _rank_df[y])
rank_label = f'$ \hat{{y}}={_rank_reg_res.intercept:.3f} + {_rank_reg_res.slope:.3f}x $'

if _col_wrap is None:
    _rank_axes = g.axes[0, 0]
    _rate_axes = g.axes[0, 1]
else:
    _rank_axes = g.axes[0]
    _rate_axes = g.axes[1]

rank_line, = _rank_axes.plot(_rank_df[x], _rank_reg_res.intercept + _rank_reg_res.slope * _rank_df[x],
                             color='#224F62', label=rank_label)

_rate_reg_res = stats.linregress(_rate_df[x], _rate_df[y])
rate_label = f'$ \hat{{y}}={_rate_reg_res.intercept:.3f} + {_rate_reg_res.slope:.3f}x $'
rate_line, = _rate_axes.plot(_rate_df[x], _rate_reg_res.intercept + _rate_reg_res.slope * _rate_df[x],
                             color='#224F62', label=rate_label)

for ax in g.axes.flat:
    ax.minorticks_on()

g.map(plt.grid, b=True, which='major', axis='both', zorder=1, linewidth=1, ms=0)
g.map(plt.grid, b=True, which='minor', axis='both', zorder=0, alpha=0.3, linewidth=0.5, ms=0)

_rank_axes.text(.038, 0.92, rank_label, transform=_rank_axes.transAxes, rotation=0, size='small')
_rate_axes.text(.038, 0.92, rate_label, transform=_rate_axes.transAxes, rotation=0, size='small')

for ax in g.axes.flat:
    ax.set_ylabel(y)
for l in ax.lines + ax.collections:
    l.set_zorder(5)

g.set_titles("{col_name}")
plt.subplots_adjust(wspace=0.2)

g.legend.set_frame_on(True)
frame = g.legend.get_frame()
frame.set_facecolor('white')
frame.set_alpha(1)

plt.savefig('plots/regplot_per_task.pdf', bbox_inches='tight')

print('Ranking reg:\n', [f'{k}={v:.2g}' for v, k in zip(_rank_reg_res, _rank_reg_res._fields)])
print('Rating reg:\n', [f'{k}={v:.2g}' for v, k in zip(_rate_reg_res, _rate_reg_res._fields)])


In [ ]:
# get coeffs of linear fit
_df = qdf_filtered.melt(id_vars=[f'nDCG@{k}', 'Topic'], value_vars=['Rating', 'Ranking'],
                        value_name="User's mean").sort_values('Topic')

for df in [_rate_df, _rank_df]:
    slope, intercept, r_value, p_value, std_err = stats.linregress(df[x], df[y])
    print(slope)

In [ ]:
# qdf_filtered.assign(topic=qdf_filtered.index.map(comb_df['topic']))
qdf_filtered.head()


In [ ]:
qdf_filtered.melt(id_vars=[f'nDCG@{k}', 'Topic'], value_vars=['Rating', 'Ranking'], value_name="User's mean")

In [ ]:
_comb_df = comb_df.reset_index().set_index(['method', 'topic', 'rid']).merge(
    all_user_queries.set_index(['method', 'topic', 'rid'])[[f'user_nDCG@{k}', 'user_query']], left_index=True,
    right_index=True)
_comb_df = _comb_df.reset_index()
comb_df = _comb_df
comb_df

In [ ]:
# sns.regplot(data=df[['user_nDCG@10', 'selected_nDCG']], x='user_nDCG@10', y='selected_nDCG')
# _df = df

In [ ]:
qpp_df = pd.read_csv('data/qpp_sd_top-50_survey.qpp', names=['qid', 'sd-60'], header=0, index_col='qid')
qpp_df.head()
qdf_filtered = qdf_filtered.assign(qpp=qpp_df)
qdf_filtered.head()

In [ ]:
print('Pearson:')
print(qdf_filtered.corr(method='pearson').to_markdown(floatfmt='.2f'))
print('\nSpearman:')
print(qdf_filtered.corr(method='spearman').to_markdown(floatfmt='.2f'))
print('\nKendall:')
print(qdf_filtered.corr(method='kendall').to_markdown(floatfmt='.2f'))

In [ ]:
qdf_filtered.head()

In [ ]:
sns.pairplot(qdf_filtered[['nDCG@10', 'Rating', 'Ranking', 'qpp']], kind='reg')
plt.savefig('plots/pairsplot.pdf')